## **Exploração e análise de dados com SQL**

#### Os dados representam informações de clientes de um banco e contam com as seguintes colunas:

- `idade`: Idade do cliente  

- `sexo`: Gênero (M/F)  
- `dependentes`: Número de dependentes  
- `escolaridade`: Nível de educação  
- `estado_civil`: Estado civil  
- `media_salario_anual`: Média salarial no ano  
- `tipo_cartao`: Tipo de cartão de crédito  
- `meses_de_relacionamento`: Tempo de relacionamento com o banco (em meses)  
- `qtd_produtos`: Quantidade de produtos adquiridos  
- `iteracoes_12m`: Interações nos últimos 12 meses  
- `meses_inativo_12m`: Meses inativos nos últimos 12 meses  
- `limite_credito`: Limite de crédito do cliente  
- `valor_transacoes_12m`: Valor total das transações nos últimos 12 meses  
- `qtd_transacoes_12m`: Quantidade de transações nos últimos 12 meses  


A tabela foi criada no **AWS Athena** junto com o **S3 Bucket** com uma versão dos dados disponibilizados em:  

github.com/andre-marcos-perez ([link](https://github.com/andre-marcos-perez/ebac-course-utils/blob/main/dataset/credito.csv))

#### **Exploração e limpeza dos dados com Python:**

Biblioteca utilizada.

In [1]:
import pandas as pd

Carregando dados.

In [2]:
URL = 'https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/credito.csv'

data = pd.read_csv(URL, sep=',')

O arquivo contém **10.127** linhas e **16** colunas.

In [3]:
data.shape

(10127, 16)

Utilizando `data.info()`, podemos ver cada coluna e seu tipo, além da informação sobre valores nulos, que é uma informação falsa.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       10127 non-null  int64 
 1   default                  10127 non-null  int64 
 2   idade                    10127 non-null  int64 
 3   sexo                     10127 non-null  object
 4   dependentes              10127 non-null  int64 
 5   escolaridade             10127 non-null  object
 6   estado_civil             10127 non-null  object
 7   salario_anual            10127 non-null  object
 8   tipo_cartao              10127 non-null  object
 9   meses_de_relacionamento  10127 non-null  int64 
 10  qtd_produtos             10127 non-null  int64 
 11  iteracoes_12m            10127 non-null  int64 
 12  meses_inativo_12m        10127 non-null  int64 
 13  limite_credito           10127 non-null  object
 14  valor_transacoes_12m     10127 non-nul

Foi removido colunas que não temos interesse.

In [5]:
data = data.drop(columns=['id', 'default'])

Utilizando `data.head()` podemos verificar na linha 4 em `estado_civil` o valor **'na'** 
representa um valor nulo mas está em formato de *string*.

In [6]:
data.head()

,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42
1,49,F,5,mestrado,solteiro,menos que $40K,blue,44,6,2,1,"8.256,96","1.291,45",33
2,51,M,3,mestrado,casado,$80K - $120K,blue,36,4,0,1,"3.418,56","1.887,72",20
3,40,F,4,ensino medio,na,menos que $40K,blue,34,3,1,4,"3.313,03","1.171,56",20
4,40,M,3,sem educacao formal,casado,$60K - $80K,blue,21,5,0,1,"4.716,22","816,08",28


Com a função `data.replace()` substituímos a *string* **'na'** por **NA**, valor nulo do pandas, por fim dropamos os valores nulos.

In [7]:
data = data.replace('na', pd.NA).dropna()

Convertemos os valores das colunas 'limite_credito' e 'valor_transacoes_12m' para *float*.
- Substituímos todos os pontos (.) antes de 3 dígitos + vírgula (,) por nada ('').
- Substituímos as vírgulas (,) por pontos (.).
- Por último, convertemos o tipo de *string* para *float*.

In [8]:
data['limite_credito'] = data['limite_credito'].str.replace(r'\.(?=\d{3},)', '', regex=True)
data['limite_credito'] = data['limite_credito'].str.replace(',', '.', regex=False)
data['limite_credito'] = data['limite_credito'].astype(float)

data['valor_transacoes_12m'] = data['valor_transacoes_12m'].str.replace(r'\.(?=\d{3},)', '',regex=True)
data['valor_transacoes_12m'] = data['valor_transacoes_12m'].str.replace(',', '.', regex=True)
data['valor_transacoes_12m'] = data['valor_transacoes_12m'].astype(float)

Após tratar os dados removendo valores nulos e colunas que não temos interesse  
o data ficou com **7081** linhas e **14** colunas.

In [9]:
data.shape

(7081, 14)

Decidi trocar a faixa salarial, que era uma *string*, por média salarial *float*.
- Removi símbolos usando `.replace()` e substituí **'K'**, que significa mil, por **'000'**.
- Condição `if` para trocar valores menores que **40k** por **20.000**.
- Outra condição para somarmos os ranges de salário e dividir por 2, tirando assim a média.
- Se o salário não cair em nenhuma condição, retornamos o valor em *float* do salário.
- Por fim, aplicamos a função criada na coluna `salario_anual` do nosso DataFrame.


In [10]:
def converter_salario(salario):
    salario = salario.replace('$', '').replace('+', '').replace('K', '000')
    
    if 'menos que' in salario:
        return 20000.0
    
    faixa = salario.split(' - ')
    if len(faixa) == 2:
        return (float(faixa[0]) + float(faixa[1])) / 2
    
    return float(salario)


data['salario_anual'] = data['salario_anual'].apply(converter_salario)

Renomeamos a coluna para representar o valor contido nela.

In [11]:
data = data.rename(columns={'salario_anual': 'media_salario_anual'})

Podemos visualizar as colunas com seus tipos corrigidos e sem valores nulos.  
Saímos de um uso de **1.2+ MB** para **829.8+ KB** em apenas **10.000** linhas.  
Se o arquivo fosse maior, o desempenho seria ainda mais significativo.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7081 entries, 0 to 10126
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   idade                    7081 non-null   int64  
 1   sexo                     7081 non-null   object 
 2   dependentes              7081 non-null   int64  
 3   escolaridade             7081 non-null   object 
 4   estado_civil             7081 non-null   object 
 5   media_salario_anual      7081 non-null   float64
 6   tipo_cartao              7081 non-null   object 
 7   meses_de_relacionamento  7081 non-null   int64  
 8   qtd_produtos             7081 non-null   int64  
 9   iteracoes_12m            7081 non-null   int64  
 10  meses_inativo_12m        7081 non-null   int64  
 11  limite_credito           7081 non-null   float64
 12  valor_transacoes_12m     7081 non-null   float64
 13  qtd_transacoes_12m       7081 non-null   int64  
dtypes: float64(3), int64(7), obj

In [13]:
data.head()

,idade,sexo,dependentes,escolaridade,estado_civil,media_salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,45,M,3,ensino medio,casado,70000.0,blue,39,5,3,1,12691.51,1144.90,42
1,49,F,5,mestrado,solteiro,20000.0,blue,44,6,2,1,8256.96,1291.45,33
2,51,M,3,mestrado,casado,100000.0,blue,36,4,0,1,3418.56,1887.72,20
4,40,M,3,sem educacao formal,casado,70000.0,blue,21,5,0,1,4716.22,816.08,28
5,44,M,2,mestrado,casado,50000.0,blue,36,3,2,1,4010.69,1088.07,24


Salvando os dados em um arquivo **CSV**.

In [14]:
data.to_csv('credito.csv', sep=',', index=False, header=False)

#### **Exploração dos dados com SQL:**

**Query:**

```sql
select * from credito_clientes limit 10;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20tudo%20from%20credito_clientes%20limit%2010.png)

---

**Query:**

```sql
describe credito_clientes;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/describe%20credito_clientes.png)

---

**Query:**

```sql
select distinct escolaridade from credito_clientes;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20district%20escolaridade%20from%20credito_clientes.png)

---

**Query:**

```sql
select distinct estado_civil from credito_clientes;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20district%20estado_civil%20from%20credito_clientes.png)

---

**Query:**

```sql
select distinct media_salario_anual from credito_clientes;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20district%20media_salario_anual%20from%20credito_clientes.png)

---

**Query:**

```sql
select count(*) as quantidade_cliente, tipo_cartao from credito_clientes group by tipo_cartao;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20count%20tudo%20as%20quantidade_cliente%20tipo_cartao%20from%20credito_cliente%20group%20by%20tipo_cartao.png)

---

**Query:**

```sql
select count(*) as quantidade_cliente, media_salario_anual from credito_clientes group by media_salario_anual order by media_salario_anual;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20count%20tudo%20%20as%20quantidade_cliente%20media_salario_anual%20from%20credito_cliente%20group%20by%20media_salario_anual.png)

---

**Query:**

```sql
select count(*) as quantidade_cliente, sexo from credito_clientes group by sexo;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20count%20tudo%20%20as%20quantidade_cliente%20sexo%20from%20credito_cliente%20group%20by%20sexo.png)

---

**Query:**

```sql
select max(limite_credito) as limite_credito, sexo from credito_clientes group by sexo order by limite_credito;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/selext%20max%20limite_credito%20sexo%20from%20credito%20clientes%20group%20by%20sexo%20order%20by%20limite_credito.png)

---

**Query:**

```sql
select count(sexo) as quantidade_cliente, sexo, media_salario_anual from credito_clientes group by media_salario_anual, sexo order by sexo;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20count%20sexo%20sexo%20media_salario_anual%20from%20credito_clientes%20group%20by%20media_salario_anual%20order%20by%20sexo.png)

---

**Query:**

```sql
select avg(limite_credito) as media_limite_credito, media_salario_anual from credito_clientes group by media_salario_anual order by media_salario_anual;
```
![](https://raw.githubusercontent.com/TheuSouza/Curso_AnalistaDeDados_EBAC/refs/heads/main/Modulo_36/img/select%20avg%20limite_credito%20media_salario_anual%20from%20credito_clientes%20group%20by%20media_salario_anual%20order%20by%20media_salario_anual.png)

---